In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor


In [3]:
## Data Source: https://www.kaggle.com/manjeetsingh/retaildataset
# read the ['Features data set.csv', 'sales data-set.csv', 'stores data-set.csv'] from the /data folder
import os
print("current directory is : " + os.getcwd()) 
print('the /data folder contains:') 
print(os.listdir("./data"))

stores = pd.read_csv('./data/stores data-set.csv')
features = pd.read_csv('./data/Features data set.csv')
sales = pd.read_csv('./data/sales data-set.csv')

## I don't intend to use so instead of replacing na-s with 0 I better drop them all
markdown_cols = features.filter(like='MarkDown').columns
for column in markdown_cols:
     features = features.drop(column,axis=1)

# On features Fill na gaps forward on CPI and Unemployment, Data seems sorted by date already
# https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html
features['CPI'] = features['CPI'].fillna(method='pad')
features['Unemployment'] = features['Unemployment'].fillna(method='pad')

current directory is : C:\Users\Dragos\projects\retail_kaggle
the /data folder contains:
['Features data set.csv', 'sales data-set.csv', 'stores data-set.csv']


In [4]:
# merge left all tables, use only keys from left frame, similar to a SQL left outer join; preserve key order.
    
retail = pd.merge(sales, features, how = 'left', on = ['Store', 'Date', 'IsHoliday'])
retail = pd.merge(retail, stores, how = 'left', on = ['Store'])

# format date in datetime, we need it to extract Year and Year week later to split the data in train, test
retail['Date'] = pd.to_datetime(retail['Date'])
retail['Year'] = pd.DatetimeIndex(retail['Date']).year
retail['Month'] = pd.DatetimeIndex(retail['Date']).month

retail['Year-Week'] = retail['Date'].dt.strftime('%Y-%U')

# I noticed there are Sales figures that are negative, cleanup:
retail = retail[retail['Weekly_Sales']>=0]

# turn IsHoliday into an Integer, useful later for numeric computations
retail['IsHoliday'] = retail['IsHoliday'].astype(int)

# Sort by Date as the data is not ordered by Date
retail = retail.sort_values(by=['Date'])

# checkout the dataset we're going to use further
retail.describe()

,Store,Dept,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,CPI,Unemployment,Size,Year,Month
count,420285.000000,420285.000000,420285.000000,420285.00000,420285.000000,420285.000000,420285.000000,420285.000000,420285.000000,420285.000000,420285.000000
mean,22.195477,44.242771,16030.329773,0.07034,60.090474,3.360888,171.212152,7.960077,136749.569176,2010.968443,6.478501
std,12.787213,30.507197,22728.500149,0.25572,18.448260,0.458523,39.162280,1.863873,60992.688568,0.796893,3.324688
min,1.000000,1.000000,0.000000,0.00000,-2.060000,2.472000,126.064000,3.879000,34875.000000,2010.000000,1.000000
25%,11.000000,18.000000,2117.560000,0.00000,46.680000,2.933000,132.022667,6.891000,93638.000000,2010.000000,4.000000
50%,22.000000,37.000000,7659.090000,0.00000,62.090000,3.452000,182.350989,7.866000,140167.000000,2011.000000,6.000000
75%,33.000000,74.000000,20268.380000,0.00000,74.280000,3.738000,212.445487,8.567000,202505.000000,2012.000000,9.000000
max,45.000000,99.000000,693099.360000,1.00000,100.140000,4.468000,227.232807,14.313000,219622.000000,2012.000000,12.000000


In [5]:
# Lag data, retail data is already ordered by Date
for i in range(1,9):
    c_name = 'Sales_Lag'+ str(i)
    retail[c_name] = retail.groupby(['Store','Dept'])['Weekly_Sales'].shift(-i)
    
retail.query('Store==1 & Dept==1').head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,CPI,Unemployment,Type,...,Month,Year-Week,Sales_Lag1,Sales_Lag2,Sales_Lag3,Sales_Lag4,Sales_Lag5,Sales_Lag6,Sales_Lag7,Sales_Lag8
34,1,1,2010-01-10,20094.19,0,71.89,2.603,211.671989,7.838,A,...,1,2010-02,57258.43,16333.14,41595.55,19403.54,16241.78,22517.56,22136.64,26229.21
8,1,1,2010-02-04,57258.43,0,62.27,2.719,210.820450,7.808,A,...,2,2010-05,16333.14,41595.55,19403.54,16241.78,22517.56,22136.64,26229.21,17558.09
21,1,1,2010-02-07,16333.14,0,80.91,2.669,211.223533,7.787,A,...,2,2010-06,41595.55,19403.54,16241.78,22517.56,22136.64,26229.21,17558.09,17596.96
2,1,1,2010-02-19,41595.55,0,39.93,2.514,211.289143,8.106,A,...,2,2010-07,19403.54,16241.78,22517.56,22136.64,26229.21,17558.09,17596.96,16145.35
3,1,1,2010-02-26,19403.54,0,46.63,2.561,211.319643,8.106,A,...,2,2010-08,16241.78,22517.56,22136.64,26229.21,17558.09,17596.96,16145.35,16555.11


In [14]:
from IPython.display import clear_output, display

results_df = []
columns = ['Pred_Dt','Store','Dept','Sales_Lag1','Sales_Lag2','Sales_Lag3','Sales_Lag4','Sales_Lag5','Sales_Lag6','Sales_Lag7','Sales_Lag8']
results_df = pd.DataFrame(columns=columns) 
predict_date = '2012-30'
    
store_list = retail['Store'].unique()
for c_store in store_list:
    dept_list=retail[retail['Store']==c_store]['Dept'].unique()
    for c_dept in dept_list:
        tmp_pred = []
        for i in range(1,9):
            c_name = 'Sales_Lag'+ str(i)
            my_cols = ['Date','Year-Week','Temperature','Fuel_Price','CPI','Unemployment','Size', 'Dept', 'IsHoliday']
            X_cols = ['Temperature','Fuel_Price','CPI','Unemployment','Size', 'Dept', 'IsHoliday']
            my_cols.append(c_name)
            retail_tmp = retail.query('Store=={store} & Dept=={dept}'.format(store=c_store, dept=c_dept))[my_cols]      
            retail_tmp = retail_tmp.dropna()
            # retail_tmp.set_index(retail_tmp['Date'], inplace=True) 
            X = retail_tmp[X_cols]
            y = retail_tmp[c_name]

            clear_output(wait=True)
            print(str(c_store)+ '/' + str(c_dept) + " Shape: " + str(retail_tmp.shape))

    
            if  X.count()[0]>20:
                model =  RandomForestRegressor(n_estimators=100, criterion='mse',
                               random_state=42, n_jobs=-1)
                model.fit(X,y)
                y_pred = model.predict(X)

                # print(mean_squared_error(y, y_pred))
                # print("R^2: {0:.4f}".format(r2_score(y, y_pred)))
          
                
                # generate predictions for one 
                pred_one = retail_tmp[retail_tmp['Year-Week']==predict_date][X_cols]

 
                if pred_one.count()[0]>0: 
                    y_test_pred = model.predict(pred_one)
                   # print(y_test_pred)
               
                else:
                        y_test_pred =0
            else:
                y_test_pred =0
            
            tmp_pred.append(float(y_test_pred))    
                
        results_df = results_df.append({"Pred_Dt":predict_date,"Store": c_store,'Dept':c_dept,
                                        'Sales_Lag1':tmp_pred[0],
                                        'Sales_Lag2':tmp_pred[1],
                                        'Sales_Lag3':tmp_pred[2],
                                        'Sales_Lag4':tmp_pred[3],
                                        'Sales_Lag5':tmp_pred[4],
                                        'Sales_Lag6':tmp_pred[5],
                                        'Sales_Lag7':tmp_pred[6],
                                        'Sales_Lag8':tmp_pred[7]}
                                       , ignore_index=True)
        # print('On' + str(predict_date) + 'the predicted ' + str(c_name) + ' for Store: '+ str(c_store) + " dept " + str(c_dept) + " is:" + str(float(y_test_pred)))



20/31 Shape: (135, 10)


KeyboardInterrupt: 

In [22]:
Sales_col = results_df.columns[results_df.columns.str.contains(pat = 'Lag')]
results_df.set_index('Store')
tmp_res= results_df[Sales_col].groupby(results_df['Store']).sum(axis=1).sum(axis=1)

In [20]:
results_df

,Pred_Dt,Store,Dept,Sales_Lag1,Sales_Lag2,Sales_Lag3,Sales_Lag4,Sales_Lag5,Sales_Lag6,Sales_Lag7,Sales_Lag8
0,2012-30,20,5,32292.4764,27034.9027,34792.9502,29187.5484,37977.3732,30652.2640,29767.1346,37951.4414
1,2012-30,20,71,6662.8117,7019.4950,7688.1990,6406.0401,9650.1321,6049.4279,6445.9397,6559.7645
2,2012-30,20,72,71720.8347,70174.9096,69518.9041,71106.6646,78452.7081,73074.4998,82570.5693,93076.6051
3,2012-30,20,74,21251.5656,23310.3599,23735.5841,21394.5334,21866.0674,20293.5033,21230.6947,21587.0314
4,2012-30,20,79,40855.0129,36444.1629,37518.6253,38401.8029,41038.4719,36229.7959,35525.7378,37540.3150
5,2012-30,20,60,720.5980,598.8840,625.5200,616.5500,692.3800,621.6240,676.4580,545.1700
6,2012-30,20,67,18346.5278,14592.1432,14328.5762,14524.2887,15549.7185,14102.3244,15151.0684,13862.8037
7,2012-30,20,85,4889.7396,4363.2422,4297.2084,4704.2550,3641.9946,3719.9401,4607.0682,4913.5439
8,2012-30,20,87,51317.9957,45279.6833,48234.4770,63045.7028,51992.8702,43263.4658,45543.5445,53904.4738
9,2012-30,20,90,97325.7646,93033.1510,93353.0229,93530.1808,102328.8763,96971.5923,94446.5031,94919.0994


In [23]:
tmp_res

Store
20    1.299579e+07
dtype: float64

In [17]:
print('Prediction for each store for the next 8 weeks, as of year-week: ' + str(predict_date))
print(tmp_res.astype(int))

Prediction for each store for the next 8 weeks, as of year-week: 2012-30
0      259656
1       56481
2      609695
3      174669
4      303553
5        5097
6      120457
7       35136
8      402582
9      765908
10     324188
11     282395
12     237548
13      43787
14       3610
15      64328
16     258049
17      22439
18     176678
19      76438
20     715700
21      53760
22     102680
23      40998
24      37685
25        455
26      24832
27      26083
28     493645
29    1212812
30     207775
31     150603
32     625402
33    1374778
34     427261
35     280844
36     627102
37     149864
38     337372
39     637393
40     342888
41     409830
42      45971
43     339353
44      63811
45      44173
dtype: int32


In [80]:
print('Prediction for the entire network for the next 8 weeks, as of year-week: ' + str(predict_date))
print(tmp_res.astype(int).sum(axis=0))

Prediction for the entire network for the next 8 weeks, as of year-week: 2012-30
374178501


In [26]:
tmp_sales= retail[Sales_col].groupby(retail['Store']).sum(axis=1).sum(axis=1)

In [33]:
 retail[retail['Year-Week']==predict_date].groupby(results_df['Store'])[Sales_col].sum(axis=1)

,Sales_Lag1,Sales_Lag2,Sales_Lag3,Sales_Lag4,Sales_Lag5,Sales_Lag6,Sales_Lag7,Sales_Lag8
Store,,,,,,,,


In [39]:
 retail[(retail['Year-Week']==predict_date) & (retail['Store']==1)]['Weekly_Sales'].sum()

1439123.71

In [27]:
tmp_sales

Store
1     1.724054e+09
2     2.132084e+09
3     4.463342e+08
4     2.326477e+09
5     3.527740e+08
6     1.733228e+09
7     6.339498e+08
8     1.007995e+09
9     6.036396e+08
10    2.103134e+09
11    1.503393e+09
12    1.118562e+09
13    2.221865e+09
14    2.233014e+09
15    6.895920e+08
16    5.758497e+08
17    9.916871e+08
18    1.199886e+09
19    1.598653e+09
20    2.334447e+09
21    8.380209e+08
22    1.139094e+09
23    1.540606e+09
24    1.502893e+09
25    7.825069e+08
26    1.111696e+09
27    1.965481e+09
28    1.463576e+09
29    5.974167e+08
30    4.853938e+08
31    1.549159e+09
32    1.294572e+09
33    2.879062e+08
34    1.072351e+09
35    1.013893e+09
36    4.116530e+08
37    5.749742e+08
38    4.284948e+08
39    1.612297e+09
40    1.067989e+09
41    1.409571e+09
42    6.184844e+08
43    7.010820e+08
44    3.360073e+08
45    8.705092e+08
dtype: float64